## Setting up

In [1]:
%env WANDB_PROJECT=asr-project-nlp-part

env: WANDB_PROJECT=asr-project-nlp-part


In [2]:
# !pip install transformers datasets huggingface_hub evaluate wandb pythainlp
# !pip install sentencepiece
# !pip install thai2transformers

In [3]:
# !wandb login

In [4]:
# from huggingface_hub import notebook_login
# notebook_login()

In [5]:
seed = 42 # The answer to everything

import torch
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)

import random
random.seed(seed)

import numpy as np
np.random.seed(seed)


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

import transformers
transformers.enable_full_determinism(seed)

In [6]:
import re
import pandas as pd

import torchtext

from datasets import load_dataset

import evaluate

In [7]:
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

## Datasets

In [8]:
model_checkpoint = 'airesearch/wangchanberta-base-att-spm-uncased'

In [9]:
tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    revision='main',
    model_max_length=416
    )

tokenizer.additional_special_tokens = ['<s>NOTUSED', '</s>NOTUSED', '<_>']

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [11]:
# Placeholder
ds = load_dataset("imdb", split='train[:1000]')

In [12]:
tokenized_ds = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Model

In [14]:
import evaluate
accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {j:i for i,j in id2label.items()}

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=2, 
    id2label=id2label, 
    label2id=label2id,
)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

In [21]:
training_args = TrainingArguments(
    output_dir="asr-project-nlp-part",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    push_to_hub=False,
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds, # This is wrong should be change to the one below
    # train_dataset=tokenized_ds["train"],
    # eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=126, training_loss=2.5616735146040956e-05, metrics={'train_runtime': 164.6905, 'train_samples_per_second': 12.144, 'train_steps_per_second': 0.765, 'total_flos': 427555464960000.0, 'train_loss': 2.5616735146040956e-05, 'epoch': 2.0})

In [ ]:
# trainer.evaluate()